### OpenAI 1113 api update 

In [1]:
import os,sys
sys.path.insert(0,'../libs')
from utils import load_json,logging,exception_handler
from llm_utils import get_oai_fees
## load API Key
key = load_json('/home/chengyu.huang/project/Fund_projects/openai_key.json') 
os.environ['OPENAI_API_KEY'] = key['ChatGPT1']['API_KEY']
import openai
from openai import OpenAI

In [13]:
## define a base openai agent class 
class BSAgent():
    def __init__(self, api_key=None, 
                 model="gpt-3.5-turbo-1106", 
                 temperature=0):

        if not api_key:
            api_key = os.environ['OPENAI_API_KEY']
        self.client = OpenAI(api_key=api_key)
        self.temperature = temperature
        if model:
            self.model = model
        self.message = []

    def _add_messages_history(self, response) -> None:

        self.message.append(response["choices"][0]["message"])

    @exception_handler(error_msg='Failed with multiple retry',
                        error_return=None,
                        attempts=3,delay=5)
    def get_completion(self,
                       prompt_template, 
                       model=None,
                       temperature=None,
                       conv_history=[],
                       return_cost=False,
                       verbose=True,
                       stream=False):
        if not model:
            model = self.model
        
        if not temperature:
            temperature = self.temperature
        
        new_message = []
        if prompt_template.get('System'):
            new_message.append({"role": "system", "content": prompt_template['System']})
        if prompt_template.get('Human'):
            new_message.append({"role": "user", "content": prompt_template['Human']})
        
        conv_history.extend(new_message)
        
        if len(conv_history) == 0 :
            raise Exception('prompt template error, prompt must be a dict with with System message or Human message.')
        
        response = self.client.chat.completions.create(
                    model=model,
                    messages=conv_history,
                    temperature=temperature,
                    stream=stream
                )
        
        if not stream:
            prompt_tokens = response.usage.prompt_tokens
            completion_tokens = response.usage.completion_tokens
            this_time_price = get_oai_fees(model, prompt_tokens, completion_tokens)
            if verbose:
                logging.info(f"************|||----|||price: {this_time_price}|||----|||************")
        
        if return_cost:
            return response,this_time_price
        
        return response 
    
    def get_response_content(self,**kwargs):
        response = self.get_completion(**kwargs)
        res_msg = response.choices[0].message.content
        return res_msg


In [14]:
## initiate an llm agent
llm_agent = BSAgent(model="gpt-3.5-turbo-1106")

In [15]:
pt = {'System':'You are a helpful assistant.',
      'Human':'What is your name?'}
res = llm_agent.get_response_content(prompt_template=pt)
print(res)

I'm a language model AI created by OpenAI, and I don't have a personal name. You can just call me OpenAI Assistant. How can I assist you today?


### Use Stream

In [18]:
completion = llm_agent.get_completion(prompt_template=pt,stream=True)
for chunk in completion:
  print(chunk.choices[0].delta.content,end="")

I'm a helpful assistant here to answer your questions and assist you with tasks. You can call me "Assistant."None

### GPT4 vision preview

In [5]:
response = llm_agent.client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason=None, index=0, message=ChatCompletionMessage(content='The image shows a scenic natural landscape featuring a wooden boardwalk extending across a green grassy field. The boardwalk leads towards the horizon and seems to provide a path through the high grass, perhaps in a wetland or marsh area. The sky is partly cloudy with fluffy blue and white clouds, suggesting it is either a late afternoon or a tranquil morning. The greenery and the clear sky contribute to the serenity of the landscape. There are also some shrubs or bushes and a few trees scattered in the distance, accentuating the diverse vegetation of the area. The image exudes a sense of peace and might be used to depict topics related to nature walks, conservation areas, wetlands, or the beauty of untouched landscapes.', role='assistant', function_call=None, tool_calls=None), finish_details={'type': 'stop', 'stop': '<|fim_suffix|>'})
